# Lesson 3 solutions 1: Taxi path lengths

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import awkward as ak

## Re-reading the dataset

For brevity, let's load the dataset with only the columns we need.

In [ ]:
taxi = ak.from_parquet(
    "https://zenodo.org/records/14537442/files/chicago-taxi.parquet",
    columns=["trip.km", "trip.begin.l*", "trip.path.*"],
    row_groups=[0],
)
taxi

<br><br><br>

## Longitude, latitude → kilometers

Reproducing the conversion from longitude, latitude differences to absolute positions to kilometers.

In [ ]:
all_longitudes = taxi.trip.begin.lon + taxi.trip.path.londiff
all_latitudes = taxi.trip.begin.lat + taxi.trip.path.latdiff

all_longitudes

In [ ]:
LONGITUDE, LATITUDE = -87.66178320769112, 41.896777262111726   # mean position (center of town)
LON_TO_KM, LAT_TO_KM = 82.98452409203695, 111.07127961503745   # lon, lat → km conversion factors

In [ ]:
km_east, km_north = (
    (all_longitudes - LONGITUDE) * LON_TO_KM,
    (all_latitudes - LATITUDE) * LAT_TO_KM,
)

<br><br><br>

## Single path example for comparison

This imperative `for` loop calculation on a single path was given in the problem statement.

I'm repeating it here so that we can compare it with the final answer (always a good idea!).

In [ ]:
example_east, example_north = (
    km_east[75, -2],
    km_north[75, -2],
)

length = 0
for i in range(len(example_east) - 1):
    Δx = example_east[i + 1] - example_east[i]
    Δy = example_north[i + 1] - example_north[i]
    Δr = np.sqrt(Δx**2 + Δy**2)
    length += Δr

length

<br><br><br>

## Solution to exercise 1

Here is a one-line calculation of all the distances of all the paths, maintaining path structure.

In [ ]:
distances = ak.sum(np.sqrt((km_east[:, :, 1:] - km_east[:, :, :-1])**2 + (km_north[:, :, 1:] - km_north[:, :, :-1])**2), axis=-1)

It has the right type: only a single `var` (same as `trips.km`).

In [ ]:
distances.type.show()

Here are the values that were provided for comparison, to help you know if you got it right.

In [ ]:
distances

And here's the one distance that had been calculated with a `for` loop. It agrees.

In [ ]:
distances[75, -2]

<br><br><br>

## Solution to exercise 2

There are two ways we could do this, but let's use the padding method.

The answer to

> How would you apply this technique to _deeply_ nested lists?

for both techniques is to use `axis=-1`.

In [ ]:
pad_km_east = ak.fill_none(ak.pad_none(km_east, 2, axis=-1), 0, axis=-1)
pad_km_north = ak.fill_none(ak.pad_none(km_north, 2, axis=-1), 0, axis=-1)

The "as the crow flies" solution is similar to the main problem, except that there's no [ak.sum](https://awkward-array.org/doc/main/reference/generated/ak.sum.html) over segments. It's a single segment from item `-1` to item `0`.

In [ ]:
crow_flies = np.sqrt((pad_km_east[:, :, -1] - pad_km_east[:, :, 0])**2 + (pad_km_north[:, :, -1] - pad_km_north[:, :, 0])**2)
crow_flies